SCRAPING JUMIA FOR DISCOUNTED ITEMS

In [9]:
import requests
from bs4 import BeautifulSoup
import re

#Define your website --Jumia--
base_url = "https://www.jumia.co.ke"
all_products = "https://www.jumia.co.ke/all-products/"

#Send a HTTP GET request to all_products url using requests library
response = requests.get(all_products)

#Create a BeautifulSoup object by parsing the HTML content retrieved by requests library
soup = BeautifulSoup(response.text, "html.parser")

- Get products in the current page

In [ ]:
products = soup.find_all('article', class_ = "prd _fb col c-prd")
print(len(products))

- Get the link to each product in that page

In [ ]:
links = []
for product in products:
    links.append(base_url+product.find('a', class_ = "core")['href'])

links

- Loop through each product and get the name, price, discount, image*

In [ ]:
product_response = requests.get('https://www.jumia.co.ke//fashion-couple-canvas-low-top-lace-up-shoes-classic-casual-sneakers-black-154042530.html')
product_soup = BeautifulSoup(product_response.text, "html.parser")
sku = product_soup.find('li', class_="-pvxs").text.split(':')[1].strip()

name = product_soup.find('h1', class_="-fs20 -pts -pbxs").text.strip()
price = product_soup.find('span', class_="-b -ubpt -tal -fs24 -prxs").text
price =  int(re.search(r'(\d[\d,]*)', price).group(1).replace(',', ''))
discount = int(product_soup.find('span', class_="bdg _dsct _dyn -mls").text.replace('%',''))



details = {
    'sku' : sku,
    'name' : name,
    'price' : price,
    'discount': discount
}
details

- Loop through pages collecting links

In [ ]:
products_list = []
# base_url = 'https://www.jumia.co.ke/all-products/?page={}#catalog-listing'
for page in range(1,10):
    response = requests.get(f'https://www.jumia.co.ke/all-products/?page={page}#catalog-listing')
    products = soup.find_all('article', class_ = "prd _fb col c-prd")
    for product in products:
        products_list.append(product)
    print(f"Getting products for page {page}")
products_list

- Combine Link collection and data collection

- Create an SQL connection to help save the data into a database

In [6]:
import mysql.connector
try:
    connection = mysql.connector.connect(
        host = 'localhost',
        port = 3306,
        user = 'root',
        database = 'jumia2',
        password = 'qwerty'
    )
except Exception as e:
    print(e)
else:
    print('Successfully connected')

Successfully connected


- Create a cursor for executing queries

In [7]:
cursor = connection.cursor()
data_entry_query = f'INSERT INTO jumia_products (SKU, name, price, old_price, discount) VALUES (%s, %s,%s,%s,%s) ON DUPLICATE KEY UPDATE name = VALUES(name), price = VALUES(price), old_price = VALUES(old_price), discount = VALUES(discount);'

In [11]:
page_url = 'https://www.jumia.co.ke/all-products/?page={}#catalog-listing'
product_count = 0

for page in range(1,1000):
    url = page_url.format(page)
    response = requests.get(url)
    products_soup = BeautifulSoup(response.text, "html.parser")
    products = products_soup.find_all('article', class_ = "prd _fb col c-prd")
    for product in products:
        link = base_url+product.find('a', class_ = "core")['href']
        print(link)
        product_response = requests.get(link)

        product_soup = BeautifulSoup(product_response.text, "html.parser")
        try:
            sku = product_soup.find('li', class_="-pvxs").text.split(':')[1].strip()
            name = product_soup.find('h1', class_="-fs20 -pts -pbxs").text.strip()
            current_price = product_soup.find('span', class_="-b -ubpt -tal -fs24 -prxs").text
            current_price =  int(re.search(r'(\d[\d,]*)', current_price).group(1).replace(',', ''))
            old_price = product_soup.find('span', class_="-tal -gy5 -lthr -fs16 -pvxs -ubpt")
            if old_price is not None:
                old_price = old_price = int(re.search(r'(\d[\d,]*)', old_price.text).group(1).replace(',', ''))
            else:
                old_price = None
            discount = product_soup.find('span', class_="bdg _dsct _dyn -mls")
            if discount is not None:
                discount = int(discount.text.replace('%',''))
            else:
                discount = None
        except Exception as e:
            print(e)
        else:
            details = {
                'sku' : sku,
                'name' : name,
                'price' : current_price,
                'old_price': old_price,
                'discount': discount
            }
            values = (sku, name, current_price, old_price, discount)

            print(f"{product_count}: {details['sku']}")
            cursor.execute(data_entry_query, values)
            connection.commit()
            product_count += 1
    
            
        
    

https://www.jumia.co.ke/campo-viejo-campo-viejo-reserva-red-wine-750-ml-58537922.html
0: CA733DR0DOEJ5NAFAMZ
https://www.jumia.co.ke/huggies-unisex-pants-gold-size-3-count-36-58597341.html
1: HU412TB08K7CXNAFAMZ
https://www.jumia.co.ke/rashnik-rn-8011-200w-solar-light-167130931.html
2: RA589LB1VTC94NAFAMZ
https://www.jumia.co.ke/chivas-chivas-xv-15-years-scotch-whiskey-750ml-58537978.html
3: CH656DB1GDKUPNAFAMZ
https://www.jumia.co.ke/red-berry-rsf810-insulated-double-wall-vacuum-bottle-700ml-black-183592272.html
4: RE999HA23QYXUNAFAMZ
https://www.jumia.co.ke/inverroche-inverroche-verdant-gin-750ml-58539143.html
5: IN863DR0KCVXTNAFAMZ
https://www.jumia.co.ke/boma-sandwich-toaster-bm-722-185748991.html
6: RE783HA1ZFOTYNAFAMZ
https://www.jumia.co.ke/red-berry-rsf1001-insulated-double-wall-vacuum-flask-1l-grey.-183613784.html
7: RE999HA2GJU2ENAFAMZ
https://www.jumia.co.ke/led-down-light-100w-solar-flood-light-dusk-to-dawn-solar-panel.-126736084.html
8: LE157LB2G5IQANAFAMZ
https://www.jumi

## Get the scraped data from the database and do EDA

In [12]:
import pandas as pd
import seaborn as sns


C:\Users\victo\AppData\Local\Temp\ipykernel_17424\1550160623.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [13]:
get_data_sql = 'SELECT * FROM jumia_products'
jumia_df = pd.read_sql(get_data_sql, connection)

C:\Users\victo\AppData\Local\Temp\ipykernel_17424\1903136482.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  jumia_df = pd.read_sql(get_data_sql, connection)


In [14]:
jumia_df.head()

,SKU,name,price,old_price,discount
0,16618MW22TM06NAFAMZ,16 Jeans Women Solid Color Sashes Casual Slim ...,4271.0,7691.0,44.0
1,16618MW2PK838NAFAMZ,16 Jeans Bivigaos Spring Autumn Classic Basic ...,5107.0,8679.0,41.0
2,16618MW2RP8EANAFAMZ,16 Jeans Casual High Waist Loose Wide Leg Pant...,4970.0,8953.0,44.0
3,91933CA2ZF4LINAFAMZ,915 Generation 4PCS Rubber Snap in TPMS Valve ...,645.0,1162.0,44.0
4,91933EA1YKMDFNAFAMZ,915 Generation Tuya Computer Power Reset Switc...,3159.0,5527.0,43.0


- convert the discount column from str into int

In [15]:
jumia_df.isnull().sum()

SKU            0
name           0
price          0
old_price    121
discount     121
dtype: int64

In [16]:
jumia_df.shape

(1517, 5)